#BLG453 Computer Vision Term Project 4
Metehan Seyran 150170903<br/>
Ugur Ali Kaplan 150170042<br/>

## Read Data

In [3]:
import nibabel as nib
import numpy as np
import cv2


ground_truth = nib.load("V_seg.nii")
img = nib.load("V.nii")
data = img.get_fdata()
gt_data = ground_truth.get_fdata()

## Region Growing algorithm

In [4]:
class Queue():
  def __init__(self):
    self.queue = []
  
  def push(self, item):
    self.queue.append(item)

  def pop(self):
    self.queue.pop(0)

  def size(self):
    return len(self.queue)

  def isEmpty(self):
    return len(self.queue) == 0

  def front(self):
    return self.queue[0]

  def clear(self):
    self.queue = []

In [5]:
class RegionGrowing2D():
  def __init__(self, img, neighbour, threshold, max_seed):
    self.img = img
    self.height = img.shape[0]
    self.width = img.shape[1]
    
    if neighbour == 4:
      self.neighbour = [[1,0], [0,1], [-1,0], [0,-1]]
    elif neighbour == 8:
      self.neighbour = [[1,0], [0,1], [-1,0], [0,-1], [-1, 1], [1, -1], [1,1], [-1,-1]]
    else:
      raise "You can only enter 4 or 8"
    self.seeds = []
    self.visited = np.zeros(self.img.shape)
    self.queue = Queue()
    self.threshold = threshold
    self.max_seed = max_seed
    self.pointMean = 0
    self.checkedPoints = 0

  def BFS(self):
    """
    Traversing the points on the 3d grid by applying Breadth-first search.
    Initially, it loops over max number of seeds, then applying BFS starting from
    that seed point. 
    """
    for i in range(self.max_seed): 

      start_node = self.findSeedPoints()
      
      if start_node == []:
        break
      
      self.pointMean = self.img[start_node[0], start_node[1]]
      self.checkedPoints = 0

      self.queue.push(start_node)
      self.visited[start_node[0], start_node[1]] = 1
      
      while not self.queue.isEmpty():
        
        temp = self.queue.front()
        self.queue.pop()

        for i in range(len(self.neighbour)):
        
          new_node = [temp[0]+self.neighbour[i][0], temp[1]+self.neighbour[i][1]]
        
          if self.checkIfOut(new_node) and self.checkThreshold(temp, new_node) and not self.isVisited(new_node):

            new_point = self.img[new_node[0], new_node[1]]
            self.checkedPoints += 1
            self.pointMean = (self.pointMean * (self.checkedPoints - 1) + new_point) / self.checkedPoints

            self.queue.push(new_node)
            self.visited[new_node[0], new_node[1]] = 1


  def isVisited(self, new_node):
    """
    Check if the new node is visited, if not return true else false.
    """
    return bool(self.visited[new_node[0], new_node[1]])

  def checkThreshold(self, old_node, new_node):
    """
    Calculate mean of positive nodes. If new node is less than threshold,
    return false, else true.
    """
    new_point = self.img[new_node[0], new_node[1]]

    return (np.abs(self.pointMean - new_point) < self.threshold)

  def checkIfOut(self, new_node):
    """
    Check if candidate node is not out of bounds
    """
    if new_node[0] < 0 or new_node[1] < 0 or new_node[0] >= self.height or new_node[1] >= self.width:
      return False
    return True

  def findSeedPoints(self):
    """
    Choose the brightest not visited point
    If there is a point that is less than or equal to 0.7, 
    the function returns an empty list.
    """
    elements = list(np.unique(self.img))
    elements.reverse()
    for element in elements:
      if element <= 0.7:
        break
      x, y = np.where(self.img == element)
      if self.visited[x[0], y[0]] != 1:
        return [x[0], y[0]]
    return []

  def calculateDiceScore(self, ground_truth):
    """
    Calculate the Dice Score based in visited array and ground truth
    """
    a, _ = np.nonzero(ground_truth)
    b, _ = np.nonzero(self.visited)

    bool_gt = (ground_truth == 1)
    bool_seg = (self.visited == 1)

    intersection = (np.logical_and(bool_gt, bool_seg)) * 1.0
    c, _ = np.nonzero(intersection)
    return 2 * (len(c))/(len(a) + len(b))

In [6]:
class RegionGrowing3D():
  def __init__(self, img, neighbour, threshold, max_seed):
    self.img = img
    self.height = img.shape[0]
    self.width = img.shape[1]
    self.depth = img.shape[2]
    
    if neighbour == 6:
      self.neighbour = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0], [0, -1, 0], [0,0,-1]]
    elif neighbour == 26:
      self.neighbour = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0], [0, -1, 0], [0,0,-1], 
                        [1, 1, 0], [-1, -1, 0], [1, -1, 0], [-1, 1, 0], [1, 1, 1], [1, 1, -1], 
                        [-1, -1, 1], [-1, -1, -1], [1, -1, 1], [1, -1, -1], [-1, 1, 1], [-1, 1, -1],
                        [1, 0, 1], [1, 0, -1], [0, 1, 1], [0, 1, -1],[-1, 0, 1], [-1, 0, -1], 
                        [0, -1, 1], [0, -1, -1]]
    else:
      raise "You can only enter 4 or 8"
    self.visited = np.zeros(self.img.shape)
    self.queue = Queue()
    self.threshold = threshold
    self.max_seed = max_seed
    self.pointMean = 0
    self.checkedPoints = 0

  def BFS(self):
    """
    Traversing the points on the 3d grid by applying Breadth-first search.
    Initially, it loops over max number of seeds, then applying BFS starting from
    that seed point. 
    """
    for i in range(self.max_seed): 
      start_node = self.findSeedPoints()
      if start_node == []:
        break
      self.pointMean = self.img[start_node[0], start_node[1], start_node[2]]
      self.checkedPoints = 0
      self.queue.push(start_node)
      self.visited[start_node[0], start_node[1], start_node[2]] = 1
      while not self.queue.isEmpty():
        temp = self.queue.front()
        self.queue.pop()

        for i in range(len(self.neighbour)):
          new_node = [temp[0]+self.neighbour[i][0], temp[1]+self.neighbour[i][1], temp[2]+self.neighbour[i][2]]
          if self.checkIfOut(new_node) and self.checkThreshold(new_node) and not self.isVisited(new_node):
            
            self.checkedPoints += 1
            new_point = self.img[new_node[0], new_node[1], new_node[2]]
            self.pointMean = (self.pointMean * (self.checkedPoints - 1) + new_point) / self.checkedPoints
            
            self.queue.push(new_node)
            self.visited[new_node[0], new_node[1], new_node[2]] = 1

  def isVisited(self, new_node):
    """
    Check if the new node is visited, if not return true else false.
    """
    return bool(self.visited[new_node[0], new_node[1], new_node[2]])

  def checkThreshold(self, new_node):
    """
    Calculate mean of positive nodes. If new node is less than threshold,
    return false, else true.
    """

    new_point = self.img[new_node[0], new_node[1], new_node[2]]
    return (np.abs(self.pointMean - new_point) < self.threshold)

  def checkIfOut(self, new_node):
    """
    Check if candidate node is not out of bounds
    """

    if new_node[0] < 0 or new_node[1] < 0 or new_node[2] < 0 or new_node[0] >= self.height or new_node[1] >= self.width or new_node[2] >= self.depth:
      return False
    return True

  def findSeedPoints(self):
    """
    Choose the brightest not visited point
    If there is a point that is less than or equal to 0.7, 
    the function returns an empty list.
    """

    elements = list(np.unique(self.img))
    elements.reverse()
    for element in elements:
      if element <= 0.7:
        break
      x, y, z = np.where(self.img == element)
      
      if self.visited[x[0], y[0], z[0]] != 1:
        return [x[0], y[0], z[0]]
    return []

  def calculateDiceScore(self, ground_truth):
    """
    Calculate the Dice Score based in visited array and ground truth
    """
    a, _, _ = np.nonzero(ground_truth)
    b, _, _ = np.nonzero(self.visited)

    bool_gt = (ground_truth == 1)
    bool_seg = (self.visited == 1)

    intersection = (np.logical_and(bool_gt, bool_seg)) * 1.0
    c, _, _ = np.nonzero(intersection)
    return 2 * (len(c))/(len(a) + len(b))

## Running Region Growing segmentation

### Task 1

*   2D segmentation using region growing algorithm.
*   15 seed points
*   8 neighborhood growing




In [7]:
avg_dice = 0.0
slices1 = []
for i in range(200):
  sample = data[i].copy()
  task1 = RegionGrowing2D(sample, neighbour=8, threshold=0.125, max_seed=15)
  task1.BFS()
  slices1.append(task1.visited)
  dice = task1.calculateDiceScore(gt_data[i])
  avg_dice += dice
  # print("Slice: ", i+1, ", Dice: ",dice)
print("Average Dice score for Task 1: ", (avg_dice/200))

slices1 = np.array(slices1)
ni_img1 = nib.Nifti1Image(slices1, np.eye(4))
nib.save(ni_img1, 'task1.nii')

Average Dice score for Task 1:  0.8542356134270266


### Task 2

*   2D segmentation using region growing algorithm.
*   15 seed points
*   4 neighborhood growing

In [8]:
avg_dice2 = 0.0
slices2 = []
for i in range(200):
  sample = data[i].copy()
  task2 = RegionGrowing2D(sample, neighbour=4, threshold=0.13, max_seed=15)
  task2.BFS()
  slices2.append(task2.visited)
  dice = task2.calculateDiceScore(gt_data[i])
  avg_dice2 += dice
  # print("Slice: ", i+1, ", Dice: ",dice)
print("Average Dice score for Task 2: ", (avg_dice2/200))

slices2 = np.array(slices2)
ni_img2 = nib.Nifti1Image(slices2, np.eye(4))
nib.save(ni_img2, 'task2.nii')

Average Dice score for Task 2:  0.8427722588505522


### Task 3

*   3D segmentation using region growing algorithm.
*   5 seed points
*   26 neighborhood growing

In [9]:
task3 = RegionGrowing3D(data, neighbour=26, threshold=0.3, max_seed=5)
task3.BFS()
dice = task3.calculateDiceScore(gt_data)
print("Dice score for task 3: ", dice)

prediction1 = np.array(task3.visited)
ni_img3 = nib.Nifti1Image(prediction1, np.eye(4))
nib.save(ni_img3, 'task3.nii')

Dice score for task 3:  0.8968862462838366


### Task 4

*   3D segmentation using region growing algorithm.
*   5 seed points
*   6 neighborhood growing

In [10]:
task4 = RegionGrowing3D(data, neighbour=6, threshold=0.25, max_seed=5)
task4.BFS()
dice = task4.calculateDiceScore(gt_data)
print("Dice score for task 4: ", dice)

prediction2 = np.array(task4.visited)
ni_img4 = nib.Nifti1Image(prediction2, np.eye(4))
nib.save(ni_img4, 'task4.nii')

Dice score for task 4:  0.8805607344299436
